In [72]:
import nibabel as nib 
import numpy as np
import logging
import matplotlib.pyplot as plt
import os
from typing import Callable

In [73]:
import nibabel as nib 
import numpy as np
import logging
import os
from typing import Callable
# TODO: Could be refactor as TFRecord class
def nii_reader(path:str, default_shape:tuple=(256, 256, 166), ignore_shape:bool=True):
    # TODO: Could be extended to multiple formats
    # TODO: Could return objects of Nibel etc.
    # TODO: Optimalize reading, test different methods 
    # https://simpleitk.readthedocs.io
    # https://nipy.org/nibael/
    # https://nilearn.github.io/
    assert os.path.isfile(path)
    img = nib.load(path)
    if img.shape != default_shape:
        logging.warning(f'Unexpected shape {img.shape}, default shape {default_shape}, file {path}')
        if ignore_shape: return None 
    
    return np.squeeze(np.array(img.get_fdata()))

def nii_dir_generator(input_dir:str,
                      fname2label: Callable[[str], str]=None,
                      image_ext:str="nii",
                      default_shape:tuple=(256, 256, 166),
                      ignore_shape:bool=False):
    for (dirpath, dirnames, filenames) in os.walk(input_dir):
        for f in filenames:
            if f.endswith(image_ext):
                f_path = os.path.join(dirpath, f)
                logging.info(f'Read nii file from {f_path}')
                img = nii_reader(f_path, default_shape=default_shape, ignore_shape=ignore_shape)
                if fname2label:
                    yield fname2label(f), img
                else:
                    yield f, img

In [74]:
import re
# ADNI tools
# TODO: Separate
def parse_adni_id(adni_img_name):
    assert adni_img_name.startswith("ADNI_")
    adni_id = re.findall(r"ADNI_([0-9]+_S_[0-9]+)_", adni_img_name)
    if len(adni_id) != 1:
        logging.error("Unknown subject ID: {}".format(adni_img_name))
    return adni_id[0]

In [124]:
# TODO: Inplace method much more faster 
# TODO: Aware of diffrent scanners 
# MARK: Sklearn does not work od 3D data
# TODO: Registration etc... https://mirtk.github.io
# https://nilearn.github.io
def normalize(data, feature_range=(0,1), method="MinMax", min_data=None, max_data=None, copy=True):
    min_out, max_out = feature_range
    if method == "MinMax":
        if not min_data: np.min(data)
        if not max_data: np.max(data)
            
        scale = (max_out - min_out) / (max_data - min_data)
        if copy: 
            return data * scale + min_out - min_data * scale
        else: 
            data *= scale
            data += min_out - min_data * scale
    else:
        raise Exception("Unknown method {}".format(method))

In [ ]:
# TODO: Check existing sollution
# tensorflow NiftyNet https://niftynet.io
# 

In [129]:
# CONFIG
# TODO: Move to config file
# IMG
# READING
IMG_PATH = 'data'
IMG_EXT = 'nii'
IMG_SHAPE = (256, 256, 166)
IMG_IGNORE_BAD_SHAPE = True
FNAME_TO_LABEL = parse_adni_id
# NORMALIZATION
NORM_METHOD = 'MinMax'
NORM_RANGE = (0, 1)
# AUGMENTATION 

# CLASS BALANCING

# TRAINING

In [128]:
# READ PHASE
filenames = []
images = []
img_generator = nii_dir_generator(input_dir=IMG_PATH,
                                  fname2label=FNAME_TO_LABEL,
                                  image_ext=IMG_EXT,
                                  default_shape=IMG_SHAPE,
                                  ignore_shape=IMG_IGNORE_BAD_SHAPE)
for fname, img in img_generator:
    filenames.append(fname)
    images.append(img)
    
images = np.array(images)

# NORMALIZATION PHASE
voxel_mean = np.mean(images)
voxel_std = np.std(images)
voxel_max = np.max(images)
voxel_min = np.min(images)

normalize(images, 
          feature_range=(0,1),
          method=NORM_METHOD, 
          min_data=voxel_min,
          max_data=voxel_max, 
          copy=False)

# DATA AUGMENTATION PHASE
# TODO: Implement

#CLASS BALANCING PHASE 
# TODO: Implement 

# TRAINING PHASE
